# Optimization and fit

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy import optimize

## Curve fitting

**Key point**: In a curve fitting, the functional form of the curve is assumed, and the curve does not necessarily pass through the data points. The curve is best fit (or "optimized") by minimizing certain error criterion.

Consider the following data sets: $X$ and $Y$, and let's pretend we don't know how the data was generated:

In [ ]:
N = 50
X = np.linspace(-5, 5, N)
Y = 2.9 * np.sin(1.5 * X) + np.random.normal(size=N)

In [ ]:
plt.plot(X,Y, 'ko')
plt.xlabel('x')
plt.ylabel('y')

The data plotted above looks like a sine wave, but we don't know the amplitude and frequency. Let's assume the data follow this function: $ f(x) = a \sin (b x) $, where $a$ is the amplitude and $b$ is the frequency. $a$ and $b$ are unknown at this point. Let's define this function in Python as:

In [ ]:
def test_func(x, a, b):
    return a * np.sin(b * x)

Now we use fit the function to the data $X$ and $Y$ using `curve_fit`. The "initial guess" for $a$ and $b$ are passed in the `p0` parameter.

In [ ]:
params, parms_cov = optimize.curve_fit(test_func, X, Y, p0=[2, 2])

The curve_fit function returns two sets of results:

`params` are the $a$ and $b$ values:

In [ ]:
params

`params_cov` are the covariance - the diagonals provide the variance of the parameter estimate (the smaller the better).

In [ ]:
parms_cov

Now we can plot the curve_fit results on top of the original data $X$, $Y$.

In [ ]:
a, b = params
a, b

In [ ]:
# use dense grid to plot smooth sine curve

plt.figure()
t = np.linspace(-5,5, 100)
plt.plot(t, test_func(t, a, b), 'r-', label='curve fit')
plt.plot(X,Y, 'ko', label='data')
plt.legend()

## Example: fitting the temperature data

The monthly Alaska temperature extremes in degrees Celcius, starting in January, are:

```
max:  17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18
min: -62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58
```

Let's first visualize the data:

In [ ]:
H = np.array([17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18])
L = np.array([-62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58])
months = np.arange(12)
plt.plot(months, H, 'ro', label='high')
plt.plot(months, L, 'bo', label='low')
plt.xlabel('month')
plt.ylabel('temperature')
plt.legend()

The distribution looks like a cosine function. Let's assume the function form of the temperature variable is a scaled, shifted cosine function:

$$avg + amp* cos[ 2\pi(t + s)/t_{max}]$$

Note the use of $s$ to shift the peak of the cosine function, and $t_{max}$ to normalize the range of the independent variable (recall the period of the cosine function is $2\pi$).

In [ ]:
def test_func(t, avg, amp, s):
    return avg + amp * np.cos( 2* np.pi * (t + s) / t.max())

In [ ]:
# fit the high temperature (H)

params_max, cov_max = optimize.curve_fit(test_func, months, H, [20, 10, 0])

days = np.linspace(0, 12, 365)
plt.plot(days, test_func(days, *params_max), 'k--')
plt.plot(months, H, 'ro')

In [ ]:
cov_max

In [ ]:
# fit the low temperature (L) using the same test_func()

params_min, cov_min = optimize.curve_fit(test_func, months, L, [-40, 20, 0])

plt.plot(days, test_func(days, *params_min), 'k--')
plt.plot(months, L, 'bo')

# also plot the high temperatures
plt.plot(days, test_func(days, *params_max), 'k--')
plt.plot(months, H, 'ro')
plt.ylabel('temperature')
plt.xlabel('month')

## Finding the minimum of a function

Consider the function: $ x^2 + 10\sin(x) $

In [ ]:
def func(x):
    return 10*np.sin(x) + x**2

# plot the function
x = np.linspace(-10,10,201)
plt.plot(x,func(x),'k-')
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x)$')

In [ ]:
# find the minimum, starting from x=x0 (initial guess)
xinit = np.array([2.5])
res = optimize.minimize(func, x0=xinit)

# plot the results
plt.plot(x,func(x),'k-')
plt.plot(xinit, func(xinit), 'go', label='initial guess')
plt.plot(res.x, func(res.x), 'ro', label='minimum')
plt.legend()

In [ ]:
# the optimization result object
res

Exercise:
    
Change the value of `xinit` (initial guess) and observe the final minimum point. For example, set `xinit` to 2.5.

## Local minimum vs global minimum

The default method of `optimize.minimize()` is `BFGS` which can only find local minimum, i.e. the solution may depend on the initial guess, as observed above. We need to use another routine `basinhopping()` to find the global minimum.

Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

In [ ]:
# find global minimum

xinit = np.array([5])
res = optimize.basinhopping(func, xinit, stepsize=2)

In [ ]:
plt.plot(x,func(x),'k-')
plt.plot(xinit, func(xinit), 'go', label='initial guess')
plt.plot(res.x, func(res.x), 'ro', label='minimum')
plt.legend()

In [ ]:
# the optimization result object contains information about the iterative optimization process
res

In [ ]:
x_glomin = res.x
x_glomin

**Note:** `basinhopping()` is not bullet-proof, e.g. if the parameters are not set correct, it can still miss the global minimum. For example:

In [ ]:
# Too small stepsize in basinhopping() results in finding local minimum that is not global minimum
xinit = np.array([5])
res = optimize.basinhopping(func, xinit, stepsize=1)
plt.plot(x,func(x),'k-')
plt.plot(xinit, func(xinit), 'go', label='initial guess')
plt.plot(res.x, func(res.x), 'ro', label='minimum')
plt.plot(x_glomin, func(x_glomin), 'ko', label='global min.')
plt.legend()

## Constrained optimization

We can limit the range of the independent variable (e.g. $x$) when finding the minimum.

Consider finding the minimum in the interval $x \in [0,10]$.

Note that the `bounds` parameter is a list of `(min, max)` pairs.

In [ ]:
xinit = np.array([1])
bd = (0, 7)
res = optimize.minimize(func, x0=xinit, bounds=[bd])
res.x

In [ ]:
fig, ax = plt.subplots()
plt.plot(x,func(x),'k-')
plt.plot(xinit, func(xinit), 'go', label='initial guess')
plt.plot(res.x, func(res.x), 'ro', label='minimum')
ax.axvspan(bd[0], bd[1], alpha=0.1, color='green')
plt.legend()

**Quiz:** Why is the minimum at $x=0$, which is not a local minimum of the function?

## Minimum of 2D functions

Consider $ f(x,y) = (4 - 2.1x^2 + \frac{x^4}{3})x^2 + xy + (4y^2 - 4)y^2 $

In [ ]:
# note: x is a 2-vector representing (x,y) coordinates

def func(x):
    f = (4 - 2.1*x[0]**2 + x[0]**4 / 3.) * x[0]**2 + x[0] * x[1] + (-4 + 4*x[1]**2) * x[1] **2
    return f

In [ ]:
x = np.linspace(-2,2,101)
y = np.linspace(-1,1,101)
xg,yg = np.meshgrid(x,y)
plt.figure()
plt.imshow(func([xg,yg]), extent=[-2,2,-1,1], origin='lower')
plt.colorbar()

In [ ]:
# 3D view of the function

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(xg, yg, func([xg, yg]), rstride=1, cstride=1,
                       cmap=plt.cm.jet, linewidth=0, antialiased=False)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x, y)')

In [ ]:
xinit = np.array([-0.5,-0.5])
res = optimize.minimize(func, xinit)

# plot
plt.figure()
# Show the function in 2D
plt.imshow(func([xg, yg]), extent=[-2, 2, -1, 1], origin="lower")
plt.colorbar()
plt.scatter(xinit[0], xinit[1], color='red', label='initial guess')
plt.scatter(res.x[0], res.x[1], color='green', label='minimum found')
plt.legend()

In [ ]:
# another way to plot

x = np.linspace(-2,2,101)
y = np.linspace(-1,1,101)
xg,yg = np.meshgrid(x,y)
z = func([xg, yg])
fig,ax = plt.subplots(1,1)
cp = ax.contourf(xg, yg, z, [-1,-0.5, 0,1,2,3,4], alpha=0.5)
ax.contour(cp, colors='black', alpha=0.6)
plt.scatter(xinit[0], xinit[1], color='red', label='initial guess')
plt.scatter(res.x[0], res.x[1], color='green', label='minimum found')
plt.legend(frameon=True, facecolor='white', edgecolor='black', framealpha=1)

In [ ]:
res

## Finding the roots

The intersection of the function $f(x)$ and the $y=0$ line.

Consider the same function as before: $ f(x) = 10\sin(x) + x^2$. We want to find $x$ where $f(x)=0$.

In [ ]:
def func(x):
    return 10*np.sin(x) + x**2

In [ ]:
xinit = np.array([-2])  # try: -2, 1
root = optimize.root(func, x0=xinit)

# plot the function and the root

x = np.linspace(-10,10,201)
plt.plot(x,func(x),'k-')
plt.plot(root.x, func(root.x), 'ro')
plt.xlim([-5,5])
plt.ylim([-10,20])
plt.hlines(0, plt.xlim()[0], plt.xlim()[1], colors='gray', linestyles='--', alpha=0.5)